In [1]:
import torch
inputs = torch.tensor(
    [[
       [0.43, 0.15, 0.89], # Your     
       [0.55, 0.87, 0.66], # journey  
       [0.57, 0.85, 0.64], # starts   
       [0.22, 0.58, 0.33], # with     
       [0.77, 0.25, 0.10], # one      
       [0.05, 0.80, 0.55]  # step     
    ],
    [
       [0.43, 0.15, 0.89], # His     
       [0.55, 0.87, 0.66], # journey  
       [0.57, 0.85, 0.64], # end   
       [0.22, 0.58, 0.33], # at     
       [0.77, 0.25, 0.10], # last      
       [0.05, 0.80, 0.55]  # way     
    ]
])

In [2]:
inputs.shape

torch.Size([2, 6, 3])

In [3]:
inputs.dim()

3

In [8]:
import torch.nn as nn

class MultiHeadAttention(nn.Module):
    def __init__(self,d_in,d_out,context_length,dropout,num_of_head,qkv_bias=False):
        super().__init__()
        
        assert (d_out % num_of_head == 0), "d_out must be divisible by num_of_head"
        
        self.d_out = d_out                                    # 2
        self.dropout = nn.Dropout(dropout)
        self.num_of_head = num_of_head                         # 2
        
        self.head_dim = d_out // num_of_head                # 2//2 = 1
        self.out_proj = nn.Linear(d_out, d_out)             
        
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        
        self.register_buffer(
            "mask",
            torch.triu(torch.ones(context_length,context_length), diagonal=1)
        )
        
    def forward(self, x):
        num_of_inputs, num_of_tokens, embedding_dim = x.shape           # 2, 6, 3
        
        queries = self.W_query(x)                            
        keys = self.W_key(x)
        values = self.W_value(x)
        
        queries = queries.view(num_of_inputs, num_of_tokens, self.num_of_head, self.head_dim)       # 2, 6, 2, 1 
        keys = keys.view(num_of_inputs, num_of_tokens, self.num_of_head, self.head_dim)       # 2, 6, 2, 1 
        values = values.view(num_of_inputs, num_of_tokens, self.num_of_head, self.head_dim)       # 2, 6, 2, 1 
        
        
        queries = queries.transpose(1,2)
        keys = keys.transpose(1,2)
        values = values.transpose(1,2)
        
        attention_score = queries @ keys.transpose(2,3)
        mask_bool = self.mask.bool() [:num_of_tokens, :num_of_tokens]
        
        attention_score.masked_fill_(mask_bool, -torch.inf)
        
        k_d = keys.shape[-1]
        attention_weight = torch.softmax(attention_score / k_d**0.5, dim=-1)
        attention_weight = self.dropout(attention_weight)
        
        context_vector = (attention_weight @ values).transpose(1,2)
        
        context_vector = context_vector.contiguous().view(
            num_of_inputs, num_of_tokens, self.d_out
        )
        context_vector = self.out_proj(context_vector)
        return context_vector

In [9]:
batch_size, context_length, d_in  = inputs.shape
d_out = 2

In [10]:
mha = MultiHeadAttention(d_in,d_out,context_length,0.0,num_of_head=2)

In [11]:
print(mha(inputs))

tensor([[[-0.3972,  0.1032],
         [-0.4516,  0.2240],
         [-0.4705,  0.2608],
         [-0.4918,  0.2259],
         [-0.4923,  0.1718],
         [-0.5047,  0.1928]],

        [[-0.3972,  0.1032],
         [-0.4516,  0.2240],
         [-0.4705,  0.2608],
         [-0.4918,  0.2259],
         [-0.4923,  0.1718],
         [-0.5047,  0.1928]]], grad_fn=<ViewBackward0>)


In [ ]:
inputs.shape

In [ ]:
mha.head_dimention